In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import cv2
import os
from tqdm import tqdm
from PIL import Image
import torchvision.models as models

In [0]:
!wget https://www.dropbox.com/s/pxcz2wdz04zxocq/CamVid.zip?dl=1 -O CamVid.zip
!unzip CamVid.zip

In [0]:
class BNConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 padding, stride):
        '''
        The constructor for the BNConv Class
        
        Arguments:
        - in_channels : The number of input channels to the first conv layer
        - out_channels : # of output channels for the first conv layer
        - kernel_size : the kernel size
        - padding : the padding height and width
        - stride : the stride length
        '''
        
        super(BNConv,self).__init__()
        
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        
        
        self.conv = nn.Conv2d(in_channels=self.in_channels,
                              out_channels=self.out_channels,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              stride=self.stride)
        
        self.bnorm = nn.BatchNorm2d(self.out_channels)
        
    def forward(self, x):
        '''
        The method which defines the forward pass of BNConv module
        
        Arguments:
        - x : the input to the network
        
        Returns:
        - the output of the network
        '''
        
        x = self.conv(x)
        x = self.bnorm(x)
        
        return x

In [0]:
class ConvolutionalBlock(nn.Module):
  def __init__(self, in_channels, oc_conv1, oc_conv2, oc_conv3):
    '''
    '''
    
    super(ConvolutionalBlock,self).__init__()
    
    self.conv1 = BNConv(in_channels=in_channels,
                   out_channels=oc_conv1,
                   kernel_size=1,
                   padding=1,
                   stride=1)
    
    self.conv2 = BNConv(in_channels=oc_conv1,
                        out_channels=oc_conv2,
                        kernel_size=3,
                        padding=1,
                        stride=2)
    
    self.conv3 = BNConv(in_channels=oc_conv2,
                        out_channels=oc_conv3,
                        kernel_size=1,
                        padding=1,
                        stride=1)
    
    self.convs = BNConv(in_channels=in_channels,
                        out_channels=oc_conv3,
                        kernel_size=3,
                        padding=1,
                        stride=2)
      
    self.relu = nn.ReLU()
    
  def forward(self, x):
    '''
    '''
    
    xm = self.relu(self.conv1(x))
    xm = self.relu(self.conv2(xm))
    xm = self.conv3(xm)
    
    xs = self.convs(x)
    print(xs.shape, xm.shape)
    x = xm + xs
    x = self.relu(x)
    
    return x

In [0]:
z = np.random.uniform(-1, 1, size=(128, 128, 64, 64))
z = torch.from_numpy(z).to('cuda')
cm = ConvolutionalBlock(in_channels=128, oc_conv1=128, oc_conv2=128, oc_conv3=256)
cm.to('cuda')
out = cm(z)
out.shape

In [0]:
class IdentityBlock(nn.Module):
  
  def __init__(self, ic_conv1, oc_conv1, oc_conv2, oc_conv3):
    '''
    '''
    
    super(IdentityBlock,self).__init__()
    
    self.conv1 = BNConv(in_channels=ic_conv1,
                        out_channels=oc_conv1,
                        kernel_size=1,
                        padding=1,
                        stride=1)
    
    self.conv2 = BNConv(in_channels=oc_conv1,
                        out_channels=oc_conv2,
                        kernel_size=3,
                        padding=1,
                        stride=1)
    
    self.conv3 = BNConv(in_channels=oc_conv2,
                        out_channels=oc_conv3,
                        kernel_size=1,
                        padding=1,
                        stride=1)
    
    self.relu = nn.ReLU()
    
  def forward(self, x):
    '''
    '''
    
    xm = self.relu(self.conv1(x))
    xm = self.relu(self.conv2(xm))
    xm = self.conv3(xm)
    
    x = x + xm
    x = self.relu(x)
    
    return x

In [0]:
class ResNetBlock(nn.Module):
  
  def __init__(self, in_channels, oc_conv1, oc_conv2, oc_conv3, num_layers):
    '''
    '''
    super(ResNetBlock,self).__init__()
    
    self.convblock = ConvolutionalBlock(in_channels=in_channels,
                                    oc_conv1=oc_conv1,
                                    oc_conv2=oc_conv2,
                                    oc_conv3=oc_conv3)
    
    self.idenblock = IdentityBlock(ic_conv1=in_channels,
                                   oc_conv1=oc_conv1,
                                   oc_conv2=oc_conv2,
                                   oc_conv3=oc_conv3)
    
    self.layers = []
    for _ in range(self.layers):
      layers.append(IdentityBlock(ic_conv1=ic_conv1,
                                  oc_conv2=oc_conv2,
                                  oc_conv3=oc_conv3,
                                  oc_conv4=oc_conv4))
    
  def forward(self, x):
    '''
    '''
    
    x = self.convblock(x)
    
    for _ in range(layers-1):
      x = self.idenblock(x)
      
    return x

In [0]:
class ASSP(nn.Module):
  def __init__(self,in_channels,out_channels = 256):
    super(ASSP,self).__init__()
    
    self.bn = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU()
    
    self.conv1 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 1,
                          padding = 0,
                          dilation=1)
    
    self.conv2 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 3,
                          padding = 6,
                          dilation = 6)
    
    self.conv3 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 3,
                          padding = 12,
                          dilation = 12)
    
    self.conv4 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 3,
                          padding = 18,
                          dilation = 18)
    
    self.conv5 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 1,
                          padding = 0,
                          dilation=1)
    
    self.convf = nn.Conv2d(in_channels = out_channels * 5, 
                          out_channels = out_channels,
                          kernel_size = 1,
                          padding = 0,
                          dilation=1)
    
    self.adapool = nn.AdaptiveAvgPool2d((16, 16))
   
  
  def forward(self,x):
    
    x1 = self.conv1(x)
    x1 = self.bn(x1)
    x1 = self.relu(x1)
    
    x2 = self.conv2(x)
    x2 = self.bn(x2)
    x2 = self.relu(x2)
    
    x3 = self.conv3(x)
    x3 = self.bn(x3)
    x3 = self.relu(x3)
    
    x4 = self.conv4(x)
    x4 = self.bn(x4)
    x4 = self.relu(x4)
    
    x5 = self.adapool(x)
    x5 = self.conv5(x5)
    x5 = self.bn(x5)
    x5 = self.relu(x5)
    
    print (x1.shape, x2.shape, x3.shape, x4.shape, x5.shape)
    x = torch.cat((x1,x2,x3,x4,x5), dim = 1) #channels first
    x = self.convf(x)
    x = self.bn(x)
    x = self.relu(x)
    
    return x

In [0]:
class ResNet_50 (nn.Module):
  def __init__(self, in_channels = 3, conv1_out = 64):
    super(ResNet_50,self).__init__()
    
    self.in_channels = in_channels
    
    self.conv1_out = conv1_out
    
    self.resnet_50 = models.resnet50(pretrained = True)
    
    self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = conv1_out, 
                           kernel_size=(7,7), stride = 2)
    
    self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2)
  
  def forward(self,x):
    x = self.conv1(x)
    x = self.maxpool(x)
    x = self.resnet_50.layer1(x)
    x = self.resnet_50.layer2(x)
    x = self.resnet_50.layer3(x)
    
    
    return x

In [0]:
class DeepLabv3(nn.Module):
  def __init__(self):
    super(DeepLabv3, self).__init__()
    
    
    
    self.resnet = ResNet_50()
    
    self.assp = ASSP(in_channels = 1024)
    
  def forward(self,x):
    
    x = self.resnet(x)
    x = F.interpolate(self.assp(x), scale_factor = 16)
    return x

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
a = torch.rand(2,3,256,256).float()
a = a.to(device)
model = DeepLabv3()
model.to(device)
with torch.no_grad():
  out = model(a)
  
out.shape

torch.Size([2, 256, 16, 16]) torch.Size([2, 256, 16, 16]) torch.Size([2, 256, 16, 16]) torch.Size([2, 256, 16, 16]) torch.Size([2, 256, 16, 16])


torch.Size([2, 256, 256, 256])

In [0]:
# Checking if there is any gpu available and pass the model to gpu or cpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
enet = enet.to(device)

In [0]:
def loader(training_path, segmented_path, batch_size, h=320, w=1000):
    filenames_t = os.listdir(training_path)
    total_files_t = len(filenames_t)
    
    filenames_s = os.listdir(segmented_path)
    total_files_s = len(filenames_s)
    
    assert(total_files_t == total_files_s)
    
    if str(batch_size).lower() == 'all':
        batch_size = total_files_s
    
    idx = 0
    while(1):
      # Choosing random indexes of images and labels
        batch_idxs = np.random.randint(0, total_files_s, batch_size)
            
        
        inputs = []
        labels = []
        
        for jj in batch_idxs:
          # Reading normalized photo
            img = plt.imread(training_path + filenames_t[jj])
          # Resizing using nearest neighbor method
            img = cv2.resize(img, (h, w), cv2.INTER_NEAREST)
            inputs.append(img)
          
          # Reading semantic image
            img = Image.open(segmented_path + filenames_s[jj])
            img = np.array(img)
          # Resizing using nearest neighbor method
            img = cv2.resize(img, (h, w), cv2.INTER_NEAREST)
            labels.append(img)
         
        inputs = np.stack(inputs, axis=2)
      # Changing image format to C x H x W
        inputs = torch.tensor(inputs).transpose(0, 2).transpose(1, 3)
        
        labels = torch.tensor(labels)
        
        yield inputs, labels

In [0]:
class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


class ConvBlock(nn.Module):
  def __init__(self,in_channels,out_channels,side_branch = True):
    super(ConvBlock,self).__init__()
    
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.side_branch = side_branch
    
    self.relu = nn.ReLU()
    
    self.sep_conv1 = SeparableConv2d(in_channels = self.in_channels,
                          out_channels = self.out_channels,
                          kernel_size = 3,
                          padding = 1)
    self.sep_conv2 = SeparableConv2d(in_channels = self.out_channels,
                          out_channels = self.out_channels,
                          kernel_size = 3,
                          padding = 1)
    self.sep_conv3 = SeparableConv2d(in_channels = self.in_channels,
                          out_channels = self.out_channels,
                          kernel_size = 3,
                          padding = 1,
                          stride = 2)
    self.conv4 = nn.Conv2d(in_channels = self.in_channels,
                          out_channels = self.out_channels,
                          kernel_size = 1,
                          padding = 1,
                          stride = 2)
    self.batchnorm = nn.BatchNorm2d()
      
        
  def forward(self,x):
    main = self.sep_conv1(x)    
    main = self.batchnorm(main)
    main = self.relu(main)
    main = self.sep_conv2(x)    
    main = self.batchnorm(main)
    main = self.relu(main)
    main = self.sep_conv3(x)    
    main = self.batchnorm(main)
    main = self.relu(main)
    
    if side_branch:
      side = F.relu(self.conv4(x))
      
    try:
      return side + main
    
    except:
      return x + main

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:01<00:00, 94440628.02it/s]


In [0]:
import torchvision.models as models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

mm = resnet_model.layer1.to(device)

with torch.no_grad():
  out = mm(a)

In [0]:
a = torch.rand(2,3,256,256).to(device)  
mmm = Res_50().to(device)
with torch.no_grad():
  out = mmm(a)
out.shape

torch.Size([2, 1024, 16, 16])